In [ ]:
from crewai import LLM
from langchain_ollama.llms import OllamaLLM
import json
from langchain_community.tools import DuckDuckGoSearchResults
from crewai.tools import tool
from crewai import Agent, Task
from crewai import Crew, Process
from datetime import datetime

llm = LLM(
    model = 'ollama/llama3.2',
    base_url = 'http://localhost:9090'
)

In [ ]:
@tool
def search_web_tool(query:str):
    """ Searches the web and return results."""
    search_tool = DuckDuckGoSearchResults(num_results=10, verbose=True)
    return search_tool.run(query)

In [ ]:
## Researcher Agent
guide_expert = Agent(
    role = 'Local City Guide Expert',
    goal = "Provides information on things to do in the city based on the user's interests.",
    backstory = """ A local expert with a passion for sharing the best experiences and hidden gems of their city. """,
    tools = [search_web_tool],
    max_iter = 5,
    llm = LLM(model='ollama/llama3.2', base_url = 'http://localhost:9091'),
    allow_delegation=False,
)

## City expert Agent
location_expert = Agent(
    role = 'Travel Trip Expert',
    goal = "Adapt to the user destination city language (Dutch if city in Dutch country.) Gather helpful information about to the city and city during travel.",
    backstory = """ A seasoned traveler who has explored various destination and knows the ins and outs of travel logistics. """,
    tools = [search_web_tool],
    max_iter = 5,
    llm = LLM(model='ollama/llama3.2', base_url = 'http://localhost:9091'),
    allow_delegation=False,
)

## Planner Expert Agent
planner_expert = Agent(
    role = 'Travel Planning Expert',
    goal = "Compiles all gathered information to provide a comprehensive travel plan.",
    backstory = """ 
    You are a professional guide with a passion for travel.
    An organizational wizard who can turn a list of possibilities into a seamless itinerary.
    """,
    tools = [search_web_tool],
    max_iter = 5,
    llm = LLM(model='ollama/llama3.2', base_url = 'http://localhost:9091'),
    allow_delegation=False,
)

In [ ]:
from_city = 'India'
destination_city = 'Paris'
date_from = '1st April 2026'
date_to = '10th April 2026'
interests = 'Sight seeing and indian food'

location_task = Task(
    description = f""" 
    In French: This task involves a comprehensive data collection process to provide the traveller with essential information about
    their destination. It includes researching and compiling details on various accomodation, ranging from budget-friendly hostela to
    luxury hotels, as well as estimating the cost of living in the area. The task also covers transportation options, visa
    requirements and any travel advisories that may be relevant. Also, consider the weather conditions forcast on the travel dates.
    And all the events that may be relevant to the traveller during the trip period.
    
    Travelling from : {from_city}
    Destination City : {destination_city}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules:
    1. If the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output = f""" 
    If the {destination_city} is in a French country : Respond in FRENCH.
    In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily
    living expenses and practical travel tips to ensure a smooth journey. 
    """,
    agent = location_expert,
    output_file = 'city_report.md',
)

In [ ]:
guide_task = Task(
    description = f""" 
    If the {destination_city} is in a French country : Respond in FRENCH.
    Tailored to the traveller's personal {interests} , this task focuses on creating an engaging and informative guide to the city's
    attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor
    activities that align with the user's preference such {interests}. The guide also highlights seasonal events and festivals that
    might be of interest during the traveller's visit.
    
    Destination City : {destination_city}
    Interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules:
    1. If the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output = f""" 
    If the {destination_city} is in a French country : Respond in FRENCH.
    An interactive detailed markdown report that presents a personalized itinerary of activities and attractions, complete with
    descriptions, locations and any necessary reservations or tickets. 
    """,
    agent = guide_expert,
    output_file = 'guide_report.md',
)

In [ ]:
planner_task = Task(
    description = f""" 
    This task synthesizes all collected information into a detailed introduction to the city (description of city and presentation, in
    3 paragraphes) cohesive and practical travel plan. And takes into account the traveller's schedule, preferences and budget to draft
    a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy
    navigation.
    
    Destination City : {destination_city}
    Interests : {interests}
    Arrival Date : {date_from}
    Departure Date : {date_to}

    Follow this rules:
    1. If the {destination_city} is in a French country : Respond in FRENCH.
    """,
    expected_output = f""" 
    If the {destination_city} is in a French country : Respond in FRENCH.
    A rich markdown document with emojis on each title and subtitle , that :
    In markdown format:
    # Welcome to {destination_city}:
    A 4 paragraphes markdown formatted including:
    - a curated articles of presentation of the city,
    - a breakdown of daily living expenses, and spots to visit.
    # Here's your Travel Plan to {destination_city} :
    Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the city's ]
    highlights based on the guide's recommendations.
    """,
    context = [location_task, guide_task],
    agent = planner_expert,
    output_file = 'travel_plan.md',
)

In [ ]:
def location_task(agent, from_city, destination_city, date_from, date_to):
    return Task(
        description = f""" 
        In French : This task involves a comprehensive data collection process to provide the traveller with essential information about
        their destination. It includes researching and compiling details on various accommodation, ranging from budget-friendly hostels
        to luxury hotels, as well as estimation the cost of living in the area. The task also covers transportation options, visa
        requirements and any travel advisories that may be relevant. Also, consider the weather conditions forcast on the travel dates.
        And all the events that may be relevant to the traveller during the trip period.

        Travelling from : {from_city}
        Destination City : {destination_city}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules:
        1. If the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output = f""" 
        If the {destination_city} is in a French country : Respond in FRENCH.
        In markdown format : A detailed markdown report that includes a curated list of recommended places to stay, a breakdown of daily
        living expenses and practical travel tips to ensure a smooth journey. 
        """,
        agent = agent,
        output_file = 'city_report.md',
        
)

def guide_task(agent, destination_city, interests, date_from, date_to):
    return Task(
        description = f""" 
        If the {destination_city} is in a French country : Respond in FRENCH.
        Tailored to the traveller's personal {interests} , this task focuses on creating an engaging and informative guide to the city's
        attractions. It involves identifying cultural landmarks, historical spots, entertainment venues, dining experiences, and outdoor
        activities that align with the user's preference such {interests}. The guide also highlights seasonal events and festivals that
        might be of interest during the traveller's visit.
        
        Destination City : {destination_city}
        Interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules:
        1. If the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output = f""" 
        If the {destination_city} is in a French country : Respond in FRENCH.
        An interactive detailed markdown report that presents a personalized itinerary of activities and attractions, complete with
        descriptions, locations and any necessary reservations or tickets. 
        """,
        agent = agent,
        output_file = 'guide_report.md',
)

def planner_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description = f""" 
        This task synthesizes all collected information into a detailed introduction to the city (description of city and presentation, in
        3 paragraphes) cohesive and practical travel plan. And takes into account the traveller's schedule, preferences and budget to draft
        a day-by-day itinerary. The planner also provides insights into the city's layout and transportation system to facilitate easy
        navigation.
        
        Destination City : {destination_city}
        Interests : {interests}
        Arrival Date : {date_from}
        Departure Date : {date_to}

        Follow this rules:
        1. If the {destination_city} is in a French country : Respond in FRENCH.
        """,
        expected_output = f""" 
        If the {destination_city} is in a French country : Respond in FRENCH.
        A rich markdown document with emojis on each title and subtitle , that :
        In markdown format:
        # Welcome to {destination_city}:
        A 4 paragraphes markdown formatted including:
        - a curated articles of presentation of the city,
        - a breakdown of daily living expenses, and spots to visit.
        # Here's your Travel Plan to {destination_city} :
        Outlines a daily detailed travel plan list with time allocations and details for each activity, along with an overview of the 
        city's highlights based on the guide's recommendations.
        """,
        context = context,
        agent = agent,
        output_file = 'travel_plan.md',
)

In [ ]:
location_task = location_task(
    location_expert,
    from_city,
    destination_city,
    date_from,
    date_to
)

guide_task = guide_task(
    guide_expert,
    destination_city,
    interests,
    date_from,
    date_to
)

planner_task = planner_task(
    [location_task, guide_task],
    planner_expert,
    destination_city,
    interests,
    date_from,
    date_to
)

In [ ]:
crew = Crew(
    agents = [location_expert, guide_expert, planner_expert],
    tasks = [location_task, guide_task, planner_task],
    process = Process.sequential,
    full_output = True,
    share_crew = False,
    verbose = True
)
result = crew.kickoff()